<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

In [2]:
# Parameters
input_geojson_path = "training-data/postcards/suva_central_postcard.geojson"
output_csv_path = "automated_notebook_runs/sampled_data_csvs/suva_central_postcard_sampled_data.csv"


## Step 1.1: Configure the environment

In [3]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [4]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [5]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "suva_east"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-suva_east-27072025


In [6]:
gdfs = []
postcards_path = "training-data/postcards"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [7]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

nm-efate-27072025_postcard.geojson
suva_west_postcard.geojson
paunganisu_postcard.geojson
mana-north_postcard.geojson
tulagi_postcard.geojson
serua_2_postcard.geojson
suva_central_postcard.geojson
efate_postcard.geojson
suva_east_postcard.geojson
bootless-bay_postcard.geojson
serua_postcard.geojson
korotogo_postcard.geojson
yanuca_postcard.geojson
mana-south_postcard.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [8]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [9]:
# filename = "nm-efate-27072025_postcard.geojson"

In [10]:
training = gpd.read_file(f"training-data/postcards/{site}_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [11]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [12]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 19MB
Dimensions:      (y: 980, x: 552, time: 1)
Coordinates:
  * y            (y) float64 8kB -2.037e+06 -2.037e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 4kB 3.17e+06 3.17e+06 ... 3.176e+06 3.176e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 1MB dask.array<chunksize=(1, 980, 552), meta=np.ndarray>
    red          (time, y, x) uint16 1MB dask.array<chunksize=(1, 980, 552), meta=np.ndarray>
    blue         (time, y, x) uint16 1MB dask.array<chunksize=(1, 980, 552), meta=np.ndarray>
    green        (time, y, x) uint16 1MB dask.array<chunksize=(1, 980, 552), meta=np.ndarray>
    emad         (time, y, x) float32 2MB dask.array<chunksize=(1, 980, 552), meta=np.ndarray>
    smad         (time, y, x) float32 2MB dask.array<chunksize=(1, 980, 552), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 1MB dask.array<chunksize=(1

In [13]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36973 instead
  warnings.warn(


In [14]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [15]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [16]:
scaled

<xarray.Dataset> Size: 58MB
Dimensions:      (y: 980, x: 552)
Coordinates:
  * y            (y) float64 8kB -2.037e+06 -2.037e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 4kB 3.17e+06 3.17e+06 ... 3.176e+06 3.176e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 4MB 0.3277 0.3879 0.4514 ... 0.0241 0.024 0.0239
    red          (y, x) float64 4MB 0.1093 0.106 0.0981 ... 0.0296 0.029 0.0284
    blue         (y, x) float64 4MB 0.1072 0.1042 0.1034 ... 0.0566 0.0574
    green        (y, x) float64 4MB 0.1296 0.128 0.1438 ... 0.0486 0.0486 0.0483
    emad         (y, x) float32 2MB 0.1168 0.1717 0.1759 ... 0.06377 0.06367
    smad         (y, x) float32 2MB 1.828e-07 2.115e-07 ... 5.437e-06 5.313e-06
    ...           ...
    swir16       (y, x) float64 4MB 0.2643 0.2877 0.2883 ... 0.0192 0.0191
    swir22       (y, x) float64 4MB 0.2041 0.2217 0.2218 ... 0.0173 0.0172
    coastal      (y, x) float64 4MB 0.1144 0.0971 0.0959 ... 0.056 0.0563 0.0564
    rededge1     (y, x) float64 4MB 0.1657 0.1999 0.1995 ... 0.0276 0.0275
    rededge2     (y, x) float64 4MB 0.2798 0.3521 0.3536 ... 0.0237 0.0236
    rededge3     (y, x) float64 4MB 0.3192 0.4028 0.4053 ... 0.0245 0.0244

In [17]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [18]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = scaled.blue.max().values
min = scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


In [19]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [20]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": scaled.y[: -pad],
    "x": scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
combined = scaled.copy()
combined = combined.assign(result_bands)

combined

<xarray.Dataset> Size: 126MB
Dimensions:      (y: 980, x: 552)
Coordinates:
  * y            (y) float64 8kB -2.037e+06 -2.037e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 4kB 3.17e+06 3.17e+06 ... 3.176e+06 3.176e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 4MB 0.3277 0.3879 0.4514 ... 0.0241 0.024 0.0239
    red          (y, x) float64 4MB 0.1093 0.106 0.0981 ... 0.0296 0.029 0.0284
    blue         (y, x) float64 4MB 0.1072 0.1042 0.1034 ... 0.0566 0.0574
    green        (y, x) float64 4MB 0.1296 0.128 0.1438 ... 0.0486 0.0486 0.0483
    emad         (y, x) float32 2MB 0.1168 0.1717 0.1759 ... 0.06377 0.06367
    smad         (y, x) float32 2MB 1.828e-07 2.115e-07 ... 5.437e-06 5.313e-06
    ...           ...
    homogeneity  (y, x) float32 2MB 0.8194 0.8056 0.7847 0.7847 ... nan nan nan
    energy       (y, x) float32 2MB 0.4061 0.4061 0.3993 0.4005 ... nan nan nan
    ASM          (y, x) float32 2MB 0.1649 0.1649 0.1594 0.1604 ... nan nan nan
    correlation  (y, x) float32 2MB 0.7181 0.6744 0.6331 0.6402 ... nan nan nan
    mean         (y, x) float32 2MB 2.111 2.167 2.132 2.076 ... nan nan nan nan
    entropy      (y, x) float32 2MB 2.832 2.793 2.771 2.709 ... nan nan nan nan

In [21]:
masked_combined, mask = all_masks(combined, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [22]:
masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [23]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2045655.0,3174365.0,3832,2024-01-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2045515.0,3175165.0,3832,2024-01-01
2,0.0660,0.0660,0.0605,0.0712,0.102097,0.000003,0.000019,0.0706,0.2415,0.0507,...,0.881944,0.510971,0.261092,0.732020,0.895833,2.321183,-2040825.0,3175485.0,3832,2024-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2046295.0,3171355.0,3832,2024-01-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2046285.0,3171695.0,3832,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,0.0321,0.0652,0.0841,0.1007,0.078250,0.000004,0.000022,0.0264,0.0380,0.0184,...,0.958333,0.781242,0.610340,0.733990,2.805556,1.093004,-2045805.0,3171225.0,3832,2024-01-01
363,0.0317,0.0452,0.0590,0.0678,0.061647,0.000004,0.000020,0.0235,0.0265,0.0133,...,0.895833,0.520880,0.271316,0.920841,2.368056,2.618766,-2045915.0,3170545.0,3832,2024-01-01
364,0.0340,0.0457,0.0593,0.0681,0.067131,0.000004,0.000022,0.0280,0.0298,0.0154,...,0.902778,0.584159,0.341242,0.845541,2.097222,2.224500,-2045905.0,3170525.0,3832,2024-01-01


In [24]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
2,14,0.0660,0.0660,0.0605,0.0712,0.102097,2.673748e-06,0.000019,0.0706,0.2415,...,0.881944,0.510971,0.261092,0.732020,0.895833,2.321183,-2040825.0,3175485.0,3832,2024-01-01
39,14,0.1584,0.1122,0.0898,0.1249,0.175684,1.259285e-06,0.000014,0.2371,0.1794,...,0.795833,0.441396,0.194830,0.722735,0.826389,2.967858,-2044485.0,3174295.0,3832,2024-01-01
61,8,0.4253,0.3793,0.2516,0.3282,0.200354,5.666703e-07,0.000008,0.3160,0.1467,...,0.740278,0.394307,0.155478,0.911746,4.131945,3.512860,-2046435.0,3174745.0,3832,2024-01-01
62,8,0.3548,0.3384,0.2365,0.3085,0.174686,2.930969e-07,0.000007,0.4179,0.2568,...,0.736111,0.365096,0.133295,0.957299,6.763889,3.950651,-2046425.0,3174695.0,3832,2024-01-01
63,8,0.0799,0.1875,0.1859,0.2457,0.168765,3.082928e-06,0.000017,0.0579,0.1465,...,0.794444,0.488980,0.239101,0.735956,3.347222,2.787090,-2046415.0,3174785.0,3832,2024-01-01


In [25]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

39 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [26]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [27]:
training_array=training_array[standard_schema]

In [28]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [29]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [30]:
training_array["cc_id"].dtype

dtype('int32')